## ImageJ Ops2
A scientific algorithm framework combined with a bevy of image processing algorithms

See the notebook: https://github.com/uw-loci/Notebooks/...



## Motivations 

The algorithm framework should prioritize:

1. **Ease of Use**: Image processing algorithms should be easy to access and simple to call.

2. **Portability**: The framework should extend Java's mantra of "write once, run anywhere". Algorithms should be usable as-is from any SciJava-compatible software, such as ImageJ or KNIME.

3. **Reproducibility**: Algorithms should produce consistent results across repeated calls and on all machines.

4. **Extensibility**: The framework should accommodate the addition of new algorithms as well as the enhancement of existing algorithms. 

<div style="text-align: right">Adapted from <a href=https://github.com/imagej/tutorials/blob/master/notebooks/1-Using-ImageJ/2-ImageJ-Ops.ipynb><code>imagej/tutorials</code></a></div>

### Op(eration)s
An Op is an implementation of an algorithm that abides by a set of conditions:

* Ops **must** be deterministic

* Ops **must** accept specific kinds of inputs and produce specific kinds of outputs
  * An Op can be defined on an image **of** integer values, or can be defined on a more specific type (like a byte)

`<T extends RealType<T>> Img<T> math.add(Img<T> img1, Img<T> img2)`

This is an Op that adds two images whose values exist within $ R^n $, producing another image whose values exist **within the same numerical space**.

### Flavors of Ops

#### Functions

Given inputs $i_1, i_2, ..., i_n$, produce one output $o$ 

<div style="float:left; margin-top: 3%; font-size: 75%; padding-right: 10px">Smiley Face Op:
    <ul>
        <li> Color: Red
        <li> Size: 100 pixels
    </ul>
</div>

<img src="rArrow.png" style = "float:left"> <img src="./SmileyFace.png" style = "float: left"> 

Benefits:
* Convenience - output is given to you
* [Power](https://en.wikipedia.org/wiki/Functional_programming) - functional programming avoids side effects

`<T extends RealType<T>> Img<T> math.add(Img<T> img1, Img<T> img2)`

This  Op adds two images whose values exist within $ R^n $, **producing another image** whose values exist **within the same numerical space**.

#### Computers
Given inputs $i_1, i_2, ..., i_n$ and preallocated output $o$, compute some value from $i_1, i_2, ..., i_n$ and store it in $o$ 

<div style="float:left; margin-top: 3%; font-size: 75%; padding-right: 10px">Smiley Face Op:
    <ul>
        <li> Color: Red
        <li> Size: 100 pixels
    </ul>
</div>
<img src="./MonaLisa.jpg" style = "float:left"> <img src="rArrow.png" style = "float: left"> <img src="./SmileyFace.png" style = "float:left"> 

Benefits:
* Efficiency: The output can be reused across multiple Op calls
* Extensibility: Users control output creation

`<T extends RealType<T>> void math.add(Img<T> img1, Img<T> img2, Img<T> output)`

This Op adds two images whose values exist within $ R^n $, **storing the result in a preallocated output** whose values exist **within the same numerical space**.

#### Inplaces

Given inputs $i_1, i_2, ..., i_n$, compute some value from $i_1, i_2, ..., i_n$ and store it in $i_j$ for some $1\le j\le n$.

<div style="float:left; margin-top: 3%; font-size: 75%; padding-right: 10px">Parameters:
    <ul>
        <li> Color: Red
        <li> Size: 100 pixels
    </ul>
</div>

<img src="./MonaLisa.jpg" style = "float:left"> <img src="rArrow.png" style = "float: left"> <img src="./MonaSmiles.jpg" style = "float:left"> 

Benefits:
* Extremely Space-Efficient: input image space is reused

`<T extends RealType<T>> void math.add(Img<T> io, Img<T> img2)`

This Op adds two images whose values exist within $ R^n $, **storing the result in the first image** (for C or Java users, this is the Op equivalent of `io += img2`)

## Architecture 

<div style="position: absolute; width: 25%; left: 5%; top: 0px">
    <b>SciJava Types</b>: Algorithms pertaining to type reasoning
    <ul style="padding-left: 5%">
        <li> Answers questions like:
        <ul style="font-size: 75%;padding-left: 5%">
            <li> Is a <code>Double</code> a <code>Number</code>?
            <li> Is an <code>Img&lt;ByteType&gt;</code> a <code>Number</code>?
            <li> Is an <code>Img&lt;ByteType&gt;</code> an <code>Img&lt;IntegerType&gt;</code>? 
        </ul>
    </ul>
</div>


<div style="position: absolute; width: 25%; left: 40%; top: -50px">
    <b>SciJava Ops</b>: A general-purpose framework for dealing with Ops
    <ul style="padding-left: 5%">
        <li> Finds Ops that satisfy user requests
        <li> Answers questions like:
        <ul style="font-size: 75%;padding-left: 5%">
            <li> Do I have a <code>math.add</code> Op that runs on <code>Img</code>s?
        </ul>
    </ul>
</div>
<div style="position: absolute; left: 33%; font-size: 200%; top: 0%">
    	&#10148;
</div>

<div style="position: absolute; width: 25%; left: 75%; top: -100px">
    <b>ImageJ Ops2</b>: A collection of Ops <em>pertaining to image processing</em>
    <ul style="padding-left: 5%">
        <li> Contains ported versions of all Ops from ImageJ Ops:
        <ul style="font-size: 75%;padding-left: 5%">
            <li> Difference of Gaussians
            <li> Colocalization
            <li> Thresholding
            <li style="list-style-type: none; font-size: 150%">&#8942;
        </ul>
    </ul>
</div>
<div style="position: absolute; left: 68%; font-size: 200%; top: -100%">
    	&#10148;
</div>

<div style="float: left; font-size: 200%">
    <center><img src="./opBag.png" style="width: 75%; height: 75%"></center>
    <p> <center><b>Bag of Ops</b></center>
</div>
<div style="float: left; font-size: 200%">
    <center><img src="./opMatcher.png" style="width: 75%; height: 75%"></center>
    <p> <center><b>Op Matcher</b></center>
</div>
<!-- <div style="float: left; font-size: 200%">
    <center><img src="./imageJOpBag.png" style="width: 50%; height: 50%"></center>
    <p> <center><b>ImageJ Ops2</b></center>
</div> -->




<div style="align: right; float: none; text-align: right; font-size: 50%">Icons made by <a href="https://www.flaticon.com/authors/freepik" title="Freepik">Freepik</a> from <a href="https://www.flaticon.com/" title="Flaticon">www.flaticon.com</a></div>

## Using Ops

First, let's load Ops:

In [ ]:
// Load the SciJava Ops library from the remote Maven repository.
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
%%classpath add mvn
org.jetbrains.kotlin kotlin-stdlib 1.3.71
net.imagej imagej 2.1.0
net.imagej imagej-ops2 0-SNAPSHOT

Added new repo: scijava.public


In [ ]:
// Initialize ImageJ Ops
ij = new net.imagej.ImageJ()
ops = ij.get(org.scijava.ops.OpService.class)

Ops can now be called with `ops.op(opName)`:

In [ ]:
ops.op("math.add") // We want an Op that adds
""

After specifying the name of the Op, we then provide the input objects or types:

In [ ]:
in1 = 2 as Double
in2 = 3 as Double

ops.op("math.add").input(in1, in2) // We want our Op to add 2 and 3
""

Once we have specified the inputs, we then specify the output object or type:

In [ ]:
ops.op("math.add").input(in1, in2).outType(Double.class) // We want our Op to give us a Double
""

Finally we ask Ops to find the Op and run it:

In [ ]:
ops.op("math.add").input(in1, in2).outType(Double.class).apply() // Run the op (note that in1 = 2 and in2 = 3 were the arguments to the Function)

We can run the same Op multiple times with different inputs:

In [ ]:
adder = ops.op("math.add").input(in1, in2).outType(Double.class).function() // Give us a function back 
""

In [ ]:
a = 0
b = 1

// find first 10 fibonacci numbers
fibonacci = [0, 1] // starter numbers 
for(int i in 1..8){
    sum = adder.apply(a as Double, b as Double)
    fibonacci.add(sum as Integer)
    a = b
    b = sum
}
fibonacci

Let's now run an Op on an image:

In [ ]:
import java.net.URL
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.img.Img
import org.scijava.io.location.BytesLocation
import io.scif.img.ImgOpener

openImg = {String imgURL, String name -> 
    url = new URL(imgURL)
    bytes = url.openStream().withCloseable{s -> s.readAllBytes()}

    bytesLocation = new BytesLocation(bytes, name)
    return new ImgOpener(ij.context()).openImgs(bytesLocation, new FloatType(), new io.scif.config.SCIFIOConfig().imgOpenerSetComputeMinMax(false)).get(0) 
}

In [ ]:
// open the image
img = openImg("https://imagej.net/images/blobs.gif", "blobs.gif")

// display converted input
ij.notebook().display(img)

Let's find the edges of the blobs:

In [ ]:
import net.imglib2.type.numeric.real.FloatType

filtered = ops.op("create.img").input(img).apply() // create image with same size and type as convertedInput
ops.op("filter.sobel").input(img).output(filtered).compute() // apply sobel filter on convertedInput

// display output
ij.notebook().display(filtered)

SciJava Ops and ImageJ Ops2 contain many more Ops; we can use the `help` Op to find out what options exist:

In [ ]:
ops.op("help").input("filter.addNoise", ops).outType(String.class).apply() // lookup all noise adding Ops

## Extending Ops

Ops can be written as either a class or as a field within a class; each style has benefits
* Ops written as Fields are more concise
* Ops written as Classes can depend on other Ops

### Ops as Classes

Let's first look at how to write an Op as a class:

In [ ]:
import java.util.function.BiFunction
import org.scijava.param.Parameter
import org.scijava.struct.ItemIO

@Parameter(key = "in1", itemIO = ItemIO.INPUT)
@Parameter(key = "in2", itemIO = ItemIO.INPUT)
@Parameter(key = "output", itemIO = ItemIO.OUTPUT)
class SampleOp implements BiFunction<Double, Double, Double> {
    @Override
    public Double apply(Double in1, Double in2) {
        return in1 * in2
    }
}

We then _"opify"_ the class and register it into the `OpEnvironment`:

In [ ]:
info = ops.env().opify(SampleOp.class)
ops.env().register(info, "test.groovyOp")

We can now call the Op through Ops:

In [ ]:
in1 = 5d
in2 = 3d

result = ops.op("test.groovyOp").input(in1, in2).outType(Double.class).apply()

Ops written as classes can depend on other ops by using the `@OpDependency` annotation:

In [ ]:
import java.util.function.Function
import java.util.List
import org.scijava.ops.OpDependency
import org.scijava.param.Parameter
import org.scijava.struct.ItemIO

@Parameter(key = "inputList", itemIO = ItemIO.INPUT)
@Parameter(key = "output", itemIO = ItemIO.OUTPUT)
class SampleMeanOp implements Function<List<Double>, Double> {
    
    @OpDependency(name = "math.add")
    public Function<List<Double>, Double> sumOp;
    
    @OpDependency(name = "stats.size")
    public Function<List<Double>, Double> sizeOp
    
    @Override
    public Double apply(List<Double> input) {
        return sumOp.apply(input) / sizeOp.apply(input) // mean = sum / size
    }
}

Just as before, we _"opify"_ the class and register it into the `OpEnvironment`:

In [ ]:
info = ops.env().opify(SampleMeanOp.class)
ops.env().register(info, "test.groovyMean")

We can then use our `SampleMeanOp`; Ops will automatically inject the required Op dependencies into our new Op (so long as the `OpService` knows about them)

In [ ]:
// samples = [1d, 2d, 4d] as List<Double>

// mean = ops.op("test.groovyMean").input(samples).outType(Double.class).apply()

java.util.function.BiFunction<Double, Double, Double> powOp = {Double base, Double exponent -> Math.pow(base, exponent)}
powOp.apply(2, 3)

### Ops as Fields

Now let's write an Op as a Field:

In [ ]:
import java.util.function.BiFunction
import org.scijava.plugin.Plugin
import org.scijava.ops.core.OpCollection
import org.scijava.ops.OpField

@Plugin(type = OpCollection.class) // tells Ops that this Class contains Field Ops
class OpCollection {

    @OpField(names = "test.groovyPower", params = "base, exponent") // tells Ops that this Field is an Op
    public final BiFunction<Double, Double, Double> powOp = 
        { Double base, Double exponent -> Math.pow(base, exponent)}

}

The combination of the `@Plugin` and `@OpField` annotations allows SciJava Ops to discover this Op

## Future Directions 

1. **Improve Accessibility**:
  * Expose SciJava Ops and ImageJ Ops2 in Fiji
  * Support Op autocompletion within popular IDEs
  * Further reduce code overhead when writing new Ops

2. **Op Utilities**:
  * Progress reporting, Logging
  * Op cancellation

3. **"Opify" more algorithms from our developer community**

## Resources

* SciJava Incubator Project https://github.com/scijava/incubator

* ImageJ Website https://imagej.net

* ImageJ Ops Tutorials https://github.com/imagej/tutorials

* Scientific Imaging Forum https://forum.image.sc